In [1]:
import os
import pandas as pd
import numpy as np
import re

# Function to parse a line with PDB codes and chains
def parse_pdb_line(line):
    parts = line.split()
    try:
        code_1, code_2 = parts[1], parts[4]
        chain_1, chain_2 = 'Unknown', 'Unknown'  # Default values
        if '_' in code_1:
            chain_1 = code_1.split('_')[1].replace('.pdb', '')
            code_1 = code_1.split('_')[0]
        if '_' in code_2:
            chain_2 = code_2.split('_')[1].replace('.pdb', '')
            code_2 = code_2.split('_')[0]
        return code_1, chain_1, code_2, chain_2
    except IndexError as e:
        print(f"Error processing line: {line}")
        raise e

# Function to extract values from the line with P-value, Afp-num, etc.
def parse_values_line(line):
    p_value = float(re.search(r'P-value (\S+)', line).group(1))
    afp_num = int(re.search(r'Afp-num (\d+)', line).group(1))
    identity = float(re.search(r'Identity (\S+%)', line).group(1).strip('%'))
    similarity = float(re.search(r'Similarity (\S+%)', line).group(1).strip('%'))
    return p_value, afp_num, identity, similarity

# Read the file and process it
data = []
with open('../data/homology/structural_homology.aln', 'r') as file:
    for line in file:
        if line.startswith('Align'):
            #print(line)
            code_1, chain_1, code_2, chain_2 = parse_pdb_line(line)
        if 'P-value' in line:
            p_value, afp_num, identity, similarity = parse_values_line(line)
            data.append([code_1, chain_1, code_2, chain_2, p_value, afp_num, identity, similarity])

# Create a DataFrame
df = pd.DataFrame(data, columns=['code_1', 'chain_1', 'code_2', 'chain_2', 'P-value', 'Afp-num', 'Identity (%)', 'Similarity (%)'])

# Display the DataFrame
print(df)

      code_1 chain_1 code_2 chain_2  P-value  Afp-num  Identity (%)  \
0       1A0F       A   1A23       A   0.1720    13385          4.48   
1       1A0F       A   1A43       A   0.1640     6081          1.09   
2       1A0F       A   1A5E       A   0.5270    10403          7.25   
3       1A0F       A   1A7V       A   0.0110    11388          7.08   
4       1A0F       A   1AAR       A   0.8280     3669         13.11   
...      ...     ...    ...     ...      ...      ...           ...   
54278   6G4B       A   6TQ3       A   0.3680    35101          4.04   
54279   6G4B       A   8TIM       A   0.2710    35907          5.88   
54280   6JHM       A   6TQ3       A   0.2810    41764          3.87   
54281   6JHM       A   8TIM       A   0.6040    43159          3.34   
54282   6TQ3       A   8TIM       A   0.0586    19171          4.71   

       Similarity (%)  
0               10.45  
1                6.52  
2               13.99  
3               18.58  
4               29.51  
...

In [2]:
fireprot = list(pd.read_csv('../data/inference/fireprot_mapped_preds.csv')['code'].unique())
s461 = list(pd.read_csv('../data/inference/s461_mapped_preds.csv')['code'].unique())
s669 = list(pd.read_csv('../data/inference/s669_mapped_preds.csv')['code'].unique())
q3421 = list(pd.read_csv('../data/inference/Q3421_mapped_preds.csv')['code'].unique())
ssym = list(pd.read_csv('../data/inference/Ssym_mapped_preds.csv')['wt_code'].unique())

<ipython-input-2-53dc2dd8a269>:1: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  fireprot = list(pd.read_csv('../data/inference/fireprot_mapped_preds.csv')['code'].unique())


In [3]:
datasets = ['fireprot', 's461', 's669', 'q3421', 'ssym'] #'s669', 
df['datasets_1'] = [[] for _ in range(len(df))]
df['datasets_2'] = [[] for _ in range(len(df))]

# Iterate over each dataset and update the DataFrame
for name, codes in zip(datasets, [fireprot, s461, s669, q3421, ssym]): #s669,
    for i in df.index:
        if df.at[i, 'code_1'] in codes:
            df.at[i, 'datasets_1'].append(name)
        if df.at[i, 'code_2'] in codes:
            df.at[i, 'datasets_2'].append(name)

df['datasets_1'] = df['datasets_1'].astype(str)
df['datasets_2'] = df['datasets_2'].astype(str)
df = df.loc[(df['datasets_1'].astype(str)!='[]') & (df['datasets_2'].astype(str)!='[]')]
#df.sort_values('Similarity (%)', ascending=False).head(50)

df['code_1'] = df['code_1'] + '_' + df['chain_1']
df['code_2'] = df['code_2'] + '_' + df['chain_2']
df

,code_1,chain_1,code_2,chain_2,P-value,Afp-num,Identity (%),Similarity (%),datasets_1,datasets_2
0,1A0F_A,A,1A23_A,A,0.1720,13385,4.48,10.45,"['s461', 's669']","['fireprot', 'q3421']"
1,1A0F_A,A,1A43_A,A,0.1640,6081,1.09,6.52,"['s461', 's669']","['fireprot', 'q3421']"
2,1A0F_A,A,1A5E_A,A,0.5270,10403,7.25,13.99,"['s461', 's669']","['fireprot', 'q3421']"
3,1A0F_A,A,1A7V_A,A,0.0110,11388,7.08,18.58,"['s461', 's669']",['s669']
4,1A0F_A,A,1AAR_A,A,0.8280,3669,13.11,29.51,"['s461', 's669']","['fireprot', 'q3421']"
...,...,...,...,...,...,...,...,...,...,...
54278,6G4B_A,A,6TQ3_A,A,0.3680,35101,4.04,12.11,['fireprot'],['fireprot']
54279,6G4B_A,A,8TIM_A,A,0.2710,35907,5.88,16.08,['fireprot'],['fireprot']
54280,6JHM_A,A,6TQ3_A,A,0.2810,41764,3.87,9.82,['fireprot'],['fireprot']
54281,6JHM_A,A,8TIM_A,A,0.6040,43159,3.34,7.90,['fireprot'],['fireprot']


In [4]:
from collections import defaultdict

def find_cluster(protein, assigned_clusters, threshold=0.01):
    for cluster in assigned_clusters:
        if all(similarity_matrix.at[protein, member] <= threshold for member in cluster):
            return cluster
    return None

for name, codes in zip(datasets, [fireprot, s461, s669, q3421, ssym]):
    df_cur = df.copy(deep=True).loc[df['datasets_1'].astype(str).str.contains(name)]
    df_cur = df_cur.loc[df['datasets_2'].astype(str).str.contains(name)]
    #df_cur = df_cur.loc[df['Similarity (%)']>50]
    # Create a list of all unique codes
    all_codes = set(df_cur['code_1']).union(set(df_cur['code_2']))

    # Pivot to create a similarity matrix
    similarity_matrix = df_cur.pivot(index='code_1', columns='code_2', values='P-value')

    # Reindex the DataFrame to include all codes in both rows and columns
    similarity_matrix = similarity_matrix.reindex(index=all_codes, columns=all_codes)

    # Fill NaN values with 0 and make the matrix symmetric
    similarity_matrix = similarity_matrix.fillna(0)
    similarity_matrix = similarity_matrix + similarity_matrix.T - similarity_matrix.multiply(similarity_matrix.T.gt(0))

    # Assign proteins to clusters
    clusters = defaultdict(list)
    for protein in similarity_matrix.index:
        cluster = find_cluster(protein, clusters.values())
        if cluster is not None:
            cluster.append(protein)
        else:
            clusters[len(clusters)].append(protein)

    # Convert the clusters dictionary to a list for better readability
    cluster_list = list(clusters.values())

    print(name, "protein clusters based on similarity:")
    print(len(cluster_list))
    print(cluster_list)

    data = pd.read_csv(f'../data/inference/{name}_mapped_preds.csv', index_col=0)
    if name == 's461':
        data['code'] = data.index.str[:4]
        
    data['cluster'] = 0
    i = 0
    for clus in cluster_list:
        i += 1
        for code_ in clus:
            code = code_[:4]
            chain = code_[-1]
            data.loc[(data['code']==code)&(data['chain']==chain), 'cluster'] = i
    data.to_csv(f'../data/{name}_mapped_preds_clusters.csv')
            

fireprot protein clusters based on similarity:
121
[['1E21_A', '1RTB_A', '1ONC_A'], ['1AYE_A'], ['1EL1_A', '1HFY_A', '1LZ1_A', '4LYZ_A', '1HFZ_A'], ['1CF3_A'], ['1M21_A'], ['1URK_A', '1TPK_A'], ['2O9P_A', '1HTI_A', '2WSY_A', '1I4N_A', '1YPI_A', '1TUX_A', '1WQ5_A', '1BTM_A', '1TPE_A', '8TIM_A'], ['1B0O_A', '2HMB_A', '1QJP_A', '2CBR_A', '1IFB_A', '1RBP_A'], ['1G6N_A', '1AJ3_A'], ['1CHK_A', '2BRD_A', '1BVC_A'], ['1AZP_A', '1SSO_A', '1C8C_A'], ['1HNG_A', '1TEN_A', '1TTG_A', '1BOY_A'], ['1K9Q_A'], ['1TCA_A', '1CQW_A', '3D2A_A'], ['1GUY_C', '2CHF_A', '6TQ3_A'], ['1C5G_A', '1UHG_A', '1ANT_L', '1QLP_A'], ['1OLR_A', '3WP4_A', '1H8V_A', '1BCX_A'], ['2ZTA_A', '1PGA_A', '1ROP_A'], ['1A5E_A', '1SVX_A', '1IHB_A'], ['1CLW_A'], ['1HME_A', '1UWO_A'], ['1UBQ_A', '1AYF_B', '1FXA_A', '1AAR_A', '1AYF_A', '1OTR_B'], ['1QGV_A', '1H7M_A', '2TRX_A'], ['1LUC_A', '1LBI_A'], ['451C_A', '1C52_A', '1YCC_A', '1YNR_A', '1AKK_A', '1YEA_A'], ['1EVQ_A', '1JU3_A', '1MJ5_A'], ['1H0C_A', '1AMQ_A', '6G4B_A'], ['1ANK_A', '2A

<ipython-input-4-b1d447b07298>:42: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'../data/inference/{name}_mapped_preds.csv', index_col=0)


s461 protein clusters based on similarity:
38
[['2NTE_A'], ['1N88_A'], ['1JL9_A'], ['5JXB_A'], ['2H3F_A'], ['1IV7_A', '3MON_B', '1IV9_A'], ['4HE7_A'], ['1NM1_A'], ['1BFM_A', '2ZTA_A', '1DIV_A'], ['3C2I_A'], ['1EKG_A', '3S4M_A'], ['1G3P_A'], ['2PTL_A'], ['2N7Z_A'], ['1IOJ_A'], ['1FXA_A', '1GUA_B', '1FRD_A'], ['2C9Q_A'], ['2LTB_A'], ['1H0X_A'], ['1ITM_A', '3D3B_A'], ['2WQG_A'], ['1BNL_A'], ['2M5S_A'], ['5OAQ_A', '3L15_A'], ['1FT8_A'], ['1XXN_A'], ['4BUQ_A'], ['1LVM_A'], ['1R2Y_A'], ['3BN0_A'], ['1A0F_A', '1JLV_A'], ['1BA3_A'], ['1L6H_A'], ['3DV0_I'], ['2ARF_A'], ['2HBB_A'], ['1O6X_A'], ['1J8I_A']]
s669 protein clusters based on similarity:
63
[['2DVV_A', '5VP3_A', '3S92_A'], ['1N88_A'], ['1JL9_A'], ['2H3F_A', '3FIS_A'], ['1PRE_A'], ['1F8I_A'], ['1FH5_H', '2CLR_B'], ['4HE7_A'], ['1NM1_A'], ['1EKG_A', '2ZTA_A', '3S4M_A'], ['1G3P_A'], ['2PTL_A', '2BJD_A'], ['1IOJ_A'], ['1FXA_A', '1FRD_A', '1GUA_B'], ['1N18_A', '3ECU_A', '1SPD_A'], ['1H0X_A', '3D2A_A'], ['1ITM_A', '2OUO_A', '1A7V_A'], ['1DXX

In [5]:
for name1 in datasets:
    for name2 in datasets:
        if name1 != name2:
            ds1 = pd.read_csv(f'../data/{name1}_mapped_preds_clusters.csv', index_col=0)
            ds2 = pd.read_csv(f'../data/{name2}_mapped_preds_clusters.csv', index_col=0)
            overlap = df.loc[((df['datasets_1'].str.contains(name1)) & (df['datasets_2'].str.contains(name2)) | (df['datasets_1'].str.contains(name2)) & (df['datasets_2'].str.contains(name1))) & (df['Identity (%)']>25)]
            overlapping_codes = list(overlap['code_1'].str[:4].unique()) + list(overlap['code_2'].str[:4].unique())
            overlapping_codes += list(set(ds1['code'].unique()).intersection(set(ds2['code'].unique())))
            ds1[f'{name2}_cluster'] = False
            ds2[f'{name1}_cluster'] = False
            ds1.loc[ds1['code'].isin(overlapping_codes), f'{name2}_cluster'] = True
            ds2.loc[ds2['code'].isin(overlapping_codes), f'{name1}_cluster'] = True
            ds1.to_csv(f'../data/{name1}_mapped_preds_clusters.csv')
            ds2.to_csv(f'../data/{name2}_mapped_preds_clusters.csv')

<ipython-input-5-a2fb853d1b9b>:4: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  ds1 = pd.read_csv(f'../data/{name1}_mapped_preds_clusters.csv', index_col=0)
<ipython-input-5-a2fb853d1b9b>:4: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  ds1 = pd.read_csv(f'../data/{name1}_mapped_preds_clusters.csv', index_col=0)
<ipython-input-5-a2fb853d1b9b>:4: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  ds1 = pd.read_csv(f'../data/{name1}_mapped_preds_clusters.csv', index_col=0)
<ipython-input-5-a2fb853d1b9b>:4: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  ds1 = pd.read_csv(f'../data/{name1}_mapped_preds_clusters.csv', index_col=0)
<ipython-input-5-a2fb853d1b9b>:5: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option

In [6]:
data = pd.read_csv('../data/homology/sequence_homology.tsv', sep='\t', header=None)
data = data.iloc[:, :3]
data.columns = ['source', 'target', 'identity']
data = data.loc[data['identity']>0.25]
data = data.loc[data['source']!=data['target']]
all_codes = set(data['source']).union(set(data['target']))

tmp = pd.read_csv(f'../data/q3421_mapped_preds_clusters.csv', index_col=0)
tmp = tmp[[c for c in tmp.columns if not 'overlaps' in c]]
tmp.to_csv(f'../data/q3421_mapped_preds_clusters.csv')

tmp = pd.read_csv('../data/fireprot_mapped_preds_clusters.csv', index_col=0)
tmp = tmp[[c for c in tmp.columns if not 'overlaps' in c]]
tmp.to_csv('../data/fireprot_mapped_preds_clusters.csv')

id_table = pd.DataFrame()
homo_table = pd.DataFrame()

for file1 in ['../data/external_datasets/korpm_training_data.csv', 
              '../data/external_datasets/rosetta_training_data.csv', 
              '../data/preprocessed/tsuboyama_mapped.csv', 
              '../data/preprocessed/fireprot_mapped.csv', 
              '../data/preprocessed/q3421_mapped.csv',
              '../data/preprocessed/s461_mapped.csv',
              '../data/preprocessed/ssym_mapped.csv'
              ]:  
    df_train = pd.read_csv(file1, index_col=0)
    if 'fireprot_mapped.csv' in file1:
        df_train['position'] = df_train['position'].fillna(-100000).astype(int)
        df_train['uid2'] = df_train['code'] + '_' + df_train['position'].astype(str) + df_train['mutation']
        df_train = df_train.reset_index()
        df_train = df_train.groupby('uid2').first()
    train_codes = set(df_train['code'])
    name1 = file1.split('/')[-1].split('_')[0]
    for file2 in ['../data/external_datasets/korpm_training_data.csv', 
                '../data/external_datasets/rosetta_training_data.csv', 
                '../data/preprocessed/tsuboyama_mapped.csv', 
                '../data/preprocessed/fireprot_mapped.csv', 
                '../data/preprocessed/q3421_mapped.csv',
                '../data/preprocessed/s461_mapped.csv',
                '../data/preprocessed/ssym_mapped.csv'
                ]:  
        c = 'code' if ('ssym' not in file2) else 'wt_code'
        name2 = file2.split('/')[-1].split('_')[0]
        if file1 != file2:
            overlap = set()
            df_test = pd.read_csv(file2, index_col=0)
            if 'fireprot_mapped.csv' in file2:
                df_test['position'] = df_test['position'].fillna(-100000).astype(int)
                df_test['uid2'] = df_test['code'] + '_' + df_test['position'].astype(str) + df_test['mutation']
                df_test = df_test.reset_index()
                df_test = df_test.groupby('uid2').first()
            #print(len(df_train), len(df_test))
            id_table.at[name1, name2] = len(df_train.join(df_test[[]], how='inner'))
            test_codes = set(df_test['code'])
            #cc_test = df_test.loc[df_test['code'].isin(overlap_codes)]
            for code in train_codes:
                if code in test_codes:
                    overlap.add(code)
                else:
                    odf = data.loc[(data['source']==code)|(data['target']==code)]
                    odf = odf.loc[data['source'].isin(test_codes)|data['target'].isin(test_codes)]
                    #print(odf)
                    if len(odf) > 0:
                        overlap.add(code)
            #print(name1, name2, overlap)
            homo_table.at[name1, name2] = len(df_test.loc[df_test[c].isin(overlap)])
            #print(len(df_test.loc[df_test['code'].isin(overlap)]))

            if 'q3421' in name1:
                name2_ = name2
                df_train[f'overlaps_{name2_}'] = False
                df_train.loc[df_train['code'].isin(overlap), f'overlaps_{name2_}'] = True
                tmp = pd.read_csv('../data/q3421_mapped_preds_clusters.csv', index_col=0)
                tmp = tmp.join(df_train[[f'overlaps_{name2_}']])
                tmp.to_csv('../data/q3421_mapped_preds_clusters.csv')

<ipython-input-6-851ffa800817>:12: DtypeWarning: Columns (108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv('../data/fireprot_mapped_preds_clusters.csv', index_col=0)


In [12]:
id_table

,rosetta,tsuboyama,fireprot,q3421,s461,ssym,korpm
korpm,1210.0,0.0,0.0,0.0,0.0,0.0,NaN
rosetta,NaN,0.0,0.0,0.0,0.0,0.0,1210.0
tsuboyama,0.0,NaN,1022.0,106.0,135.0,0.0,0.0
fireprot,0.0,1022.0,NaN,2244.0,49.0,170.0,0.0
q3421,0.0,106.0,2244.0,NaN,50.0,205.0,0.0
s461,0.0,135.0,49.0,50.0,NaN,0.0,0.0
ssym,0.0,0.0,170.0,205.0,0.0,NaN,0.0


In [13]:
homo_table

,rosetta,tsuboyama,fireprot,q3421,s461,ssym,korpm
korpm,956.0,13814.0,4245.0,2897.0,360.0,596.0,NaN
rosetta,NaN,2354.0,3152.0,1928.0,11.0,554.0,1406.0
tsuboyama,33.0,NaN,1031.0,130.0,163.0,0.0,901.0
fireprot,1011.0,6324.0,NaN,3310.0,71.0,382.0,1936.0
q3421,1017.0,7123.0,4896.0,NaN,71.0,618.0,1920.0
s461,4.0,4224.0,51.0,52.0,NaN,0.0,147.0
ssym,333.0,0.0,1006.0,926.0,0.0,NaN,275.0
